Sections:
1. EDA
2. NLP on whole corpus
3. NER Prototypiong
4. Sentiment Prototyping


# 1. EDA



This workbook contains mostly EDA and an introduction to the NYT corpus we are working with. In addition, we prototype the mechanisms for NER which we will use to analyse individual teams and players for further analysis.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from collections import Counter
from textblob import TextBlob, Word
import nltk
from nltk.corpus import stopwords
import string
import spacy

In [2]:
%matplotlib inline

In [3]:
articles_list = pd.read_csv('../nyt_scrape/nyt_article_list.csv')

In [4]:
articles_list['title'] = articles_list.article_urls.map(lambda x : x.split('/')[-1].split('.')[0])

In [5]:
articles_list.head()

,published_date,article_urls,article_summary,article_headline,title
0,2018-06-09T17:31:27+0000,https://www.nytimes.com/2018/06/09/sports/nba-...,"Accused of making the sport uncompetitive, the...",The Warriors Were Dominant. But How Dominant?,nba-finals-sweep
1,2018-06-09T01:26:37+0000,https://www.nytimes.com/2018/06/08/movies/kyri...,"In his most extensive comments to date, the Bo...",Kyrie Irving Doesn’t Know if the Earth Is Roun...,kyrie-irving-nba-celtics-earth
2,2018-06-08T22:00:03+0000,https://www.nytimes.com/2018/06/08/sports/nba-...,Kevin Durant was named the finals’ M.V.P. agai...,"Warriors, in Full Dynasty Mode, Sweep Cavalier...",nba-finals-warriors-cavs
3,2018-06-08T17:33:17+0000,https://www.nytimes.com/2018/06/08/sports/lebr...,"If James wants to beat the Warriors, he may ne...","LeBron James Reveals an Injury, but His Destin...",lebron-james-free-agency
4,2018-06-08T13:16:38+0000,https://www.nytimes.com/2018/06/08/sports/game...,The animated show combining elements of “Game ...,The ‘Game of Zones’ Guys Knew You Wanted a Bry...,game-of-zones


In [6]:
dir_list = os.listdir('../nyt_scrape/articles')
dir_list_split = [name.split('_')[0] for name in dir_list]

Checking that two article indices of same length:

In [7]:
assert len(articles_list) == len(dir_list_split)

Total articles:

In [8]:
print(len(articles_list))

320


Sample article:

In [9]:
with open('../nyt_scrape/articles/'+np.random.choice(dir_list)) as f:
    print(f.read()[:1000],'...')

Kristaps Porzingis scored a career-high 38 points and Kyle O’Quinn added 15 points and 12 rebounds to help the Knicks beat the Denver Nuggets, 116-110, at Madison Square Garden on Monday night.
Tim Hardaway Jr. scored all 13 of his points in the fourth quarter after the Knicks had blown all of their 23-point third-quarter lead.
Nikola Jokic led the Nuggets with 28 points and Jamal Murray scored 20.
The Knicks had taken their biggest lead of the game, 69-46, on Courtney Lee’s layup with 10 minutes 38 seconds left in the third quarter. However, the Nuggets responded by scoring the next 13 points to ignite a 27-2 run, and they took their first lead, 73-71, on Wilson Chandler’s free throws with 4:42 left in the third quarter.
The Knicks committed 10 turnovers during that six-minute stretch.
The game was tied 77-77 when Doug McDermott’s 3-pointer put the Knicks back in front for good with 1:09 left in the third.
The Knicks led 84-81 going into the final quarter.
Will Barton brought the Nugg

Parse dates:

In [10]:
articles_list['date'] = articles_list.published_date.apply(lambda x : x.split('T')[0])

In [11]:
articles_list.date = pd.to_datetime(articles_list.date)

Rewrite this to csv for convenience next time:

In [12]:
articles_list.to_csv('../nyt_scrape/articles_list_w_date.csv',index=False)

# 2. NLP on whole Corpus:

In [13]:
corpus = ''

for article in dir_list:
    with open(f'../nyt_scrape/articles/{article}') as f:
        corpus += f.read()
        corpus += '\n'

corpus = corpus.replace('’','').replace('”','').replace('“','').replace('—','')

Approximate number of words per article (assuming 6 characters per word):

In [14]:
len(corpus)/(len(dir_list)*6)

797.4786458333333

Word frequencies:

In [15]:
corpus_blob = TextBlob(corpus)

In [16]:
corpus_words = corpus_blob.words
corpus_words = [word.lower() for word in corpus_words if word.lower() not in stopwords.words('english')\
                                                         and word not in string.punctuation]
c = Counter(corpus_words)

In [17]:
c.most_common()

[('said', 1583),
 ('game', 1358),
 ('points', 962),
 ('team', 933),
 ('season', 844),
 ('n.b.a', 731),
 ('one', 700),
 ('first', 663),
 ('james', 633),
 ('players', 602),
 ('knicks', 590),
 ('two', 529),
 ('coach', 524),
 ('warriors', 521),
 ('games', 514),
 ('would', 513),
 ('like', 508),
 ('last', 479),
 ('teams', 463),
 ('time', 454),
 ('basketball', 451),
 ('new', 431),
 ('play', 404),
 ('league', 385),
 ('cavaliers', 370),
 ('player', 366),
 ('back', 351),
 ('even', 349),
 ('night', 348),
 ('made', 347),
 ('years', 346),
 ('also', 340),
 ('could', 333),
 ('get', 321),
 ('three', 308),
 ('golden', 308),
 ('going', 306),
 ('left', 304),
 ('celtics', 302),
 ('rockets', 298),
 ('quarter', 294),
 ('state', 290),
 ('rebounds', 288),
 ('much', 281),
 ('series', 280),
 ('way', 279),
 ('conference', 278),
 ('lead', 276),
 ('year', 270),
 ('scored', 264),
 ('cleveland', 264),
 ('second', 263),
 ('played', 258),
 ('finals', 258),
 ('minutes', 258),
 ('ball', 249),
 ('still', 248),
 ('mr', 24

Restricting to nouns:

In [18]:
tags = corpus_blob.pos_tags

In [19]:
tags = [tag for tag in tags if tag[1] in {'NN','NNS','NNP','NNPS'}]

In [20]:
tags = [Word(tag[0].lower()) for tag in tags]

In [21]:
lemma_tags = [tag.lemmatize() for tag in tags]
lemma_tags = [tag for tag in tags if tag not in ['s','t']]

In [22]:
c2 = Counter(lemma_tags)
c2.most_common(20)

[('game', 1350),
 ('points', 960),
 ('team', 927),
 ('season', 844),
 ('n.b.a', 679),
 ('james', 633),
 ('players', 602),
 ('knicks', 590),
 ('warriors', 521),
 ('games', 514),
 ('coach', 499),
 ('time', 454),
 ('teams', 447),
 ('basketball', 439),
 ('league', 377),
 ('cavaliers', 370),
 ('player', 366),
 ('night', 348),
 ('years', 346),
 ('celtics', 302)]

# 3. Prototype NER techniques:

In [23]:
nlp = spacy.load('en_core_web_sm')

Note spacy restricts us to using a maximum of two thirds of the corpus:

In [24]:
doc = nlp(corpus[:999999])

In [25]:
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Virginia 32 40 GPE
Julius Erving 70 83 ORG
the New York Nets 87 104 ORG
1973 108 112 DATE
Erving 114 120 ORG
first 132 137 ORDINAL
Nassau Coliseum 149 164 ORG

 254 255 GPE
Erving 273 279 ORG

 386 387 GPE
Coliseum 391 399 ORG
Erving 428 434 ORG
Long Island 469 480 LOC
Roosevelt High School 504 525 ORG
three seasons 535 548 DATE
two 634 637 CARDINAL
A.B.A. 638 644 GPE
the N.B.A. and Erving 689 710 ORG
the Philadelphia 76ers 723 745 ORG

 746 747 GPE
Erving 751 757 ORG
Long Island 782 793 LOC
this weekend 857 869 DATE
Coliseum 923 931 GPE
first 940 945 ORDINAL
decades 954 961 DATE
J 972 973 PERSON

 1008 1009 GPE
The Long Island Nets 1009 1029 FAC
the N.B.A. G League 1039 1058 ORG
the Brooklyn Nets 1072 1089 ORG
Erving 1105 1111 ORG
Don Ryan 1138 1146 PERSON
Saturdays season 1151 1167 DATE
the Fort Wayne Mad Ants 1183 1206 ORG
New York Nets 1220 1233 GPE
Ryan 1262 1266 PERSON

 1364 1365 GPE
opening night 1375 1388 TIME
Alton Byrd 1487 1497 PERSON

 1559 1560 GPE
Coliseum 1657 1665 PERS

114 16020 16023 CARDINAL
Wizards 106 16025 16036 ORG
23 16053 16055 CARDINAL
12 16067 16069 CARDINAL
Delon Wright 16080 16092 PERSON
11 16100 16102 CARDINAL
18 16110 16112 CARDINAL
the fourth quarter 16123 16141 DATE
Toronto 16152 16159 GPE
10 16178 16180 CARDINAL
Eastern Conference 16227 16245 LOC
Washington 16252 16262 GPE

 16263 16264 GPE
DeMar DeRozan 16264 16277 PERSON
17 16284 16286 CARDINAL
C. J. Miles 16295 16306 PERSON
G Anunoby 16313 16322 ORG
12 16332 16334 CARDINAL
Kyle Lowry 16340 16350 ORG
11 16355 16357 CARDINAL
9 16369 16370 CARDINAL
second 16476 16482 ORDINAL
Philadelphia 16497 16509 GPE
2001 16513 16517 DATE

 16518 16519 GPE
Toronto 16519 16526 GPE
N.B.A 16550 16555 GPE
10 16563 16565 CARDINAL
Game 16578 16582 FAC
six 16608 16611 CARDINAL

 16620 16621 GPE
DeRozan 16645 16652 PERSON
Worry about Game 2. 16673 16692 ORG

 16692 16693 GPE
the Eastern Conference 16716 16738 LOC
Toronto 16740 16747 GPE
16 16753 16755 CARDINAL
30 3-point 16759 16769 CARDINAL
Miles 16785 1

CELTICS 30984 30991 ORG
92 30992 30994 CARDINAL
Khris Middleton 30995 31010 ORG
23 31018 31020 CARDINAL
Milwaukee 31042 31051 GPE
Boston 31057 31063 GPE
two 31093 31096 CARDINAL
one 31106 31109 CARDINAL
NYT)
 31112 31117 ORG
WINSLOW 31117 31124 ORG
Miamis 31148 31154 NORP
Justise Winslow 31155 31170 PERSON
15,000 31172 31178 MONEY
Philadelphia 31232 31244 GPE
Joel Embiids 31252 31264 GPE
Thursday 31273 31281 DATE
Winslow 31283 31290 PERSON
Embiids 31316 31323 NORP
AP 31366 31368 ORG

 31369 31370 GPE
Michael Jordan 31413 31427 PERSON
LeBron James 31489 31501 PERSON
6 31528 31529 CARDINAL
3 31532 31533 CARDINAL

 31534 31535 GPE
6 31589 31590 CARDINAL
3 31593 31594 CARDINAL
Jordans 31637 31644 NORP
6 31645 31646 CARDINAL
Jamess 31677 31683 ORG
3-5 31684 31687 PERCENT
Jerry West   31825 31837 PERSON
1 31845 31846 CARDINAL

 31897 31898 GPE
first 32014 32019 ORDINAL
Saturday 32045 32053 DATE

 32216 32217 GPE
Stephen Curry 32220 32233 PERSON
the Golden State Warriors 32238 32263 GPE
the l

 51428 51429 GPE
Fuzzy Levane 51465 51477 PERSON
Ned Irish 51533 51542 PERSON
four 51563 51567 CARDINAL
African 51568 51575 NORP
one 51619 51622 CARDINAL
Ray Felix 51633 51642 PERSON
Johnny Green 51644 51656 PERSON
Willie Naulls 51661 51674 PERSON
Ramsey 51683 51689 PERSON
Syracuse 51702 51710 GPE

 51815 51816 GPE
Monday 51828 51834 DATE
night 51835 51840 TIME
Carmelo Anthony 51858 51873 PERSON
Knicks 51878 51884 ORG
29-13 second-quarter 51897 51917 TIME
Latvian 51932 51939 NORP
Porzingis 51947 51956 GPE
France 51985 51991 GPE
Frank Ntilikina 51993 52008 PERSON
Turkish 52016 52023 NORP

 52045 52046 GPE
Ramsey 52087 52093 PERSON
his day 52147 52154 DATE
Knicks 52222 52228 ORG
New York 52257 52265 GPE

 52334 52335 GPE
Anthony 52342 52349 PERSON
Knicks 52359 52365 ORG
James 52390 52395 PERSON
  52427 52428 NORP
  52446 52447 NORP
Monday 52512 52518 DATE
Jamess 52544 52550 ORG
Knicks 52577 52583 ORG
Ntilikina 52597 52606 GPE
Dallass 52612 52619 ORG
first-quarter 52668 52681 DATE
two 527

 77347 77348 GPE
Follow NYT Food 77348 77363 ORG
Instagram 77377 77386 GPE
Twitter 77388 77395 GPE
Pinterest 77400 77409 LOC
NYT Cooking 77436 77447 ORG

 77507 77508 GPE
The Golden State Warriors 77508 77533 FAC
third-quarter 77554 77567 DATE
the fourth quarter 77605 77623 DATE
their 17th 77665 77675 DATE
Oakland 77707 77714 GPE

 77715 77716 GPE
Houston 77729 77736 GPE
the Western Conference 77802 77824 FAC
this season 77825 77836 DATE
James Harden 77843 77855 PERSON
Chris Paul 77860 77870 PERSON
95 77907 77909 CARDINAL
Game 4 77924 77930 LAW
the Western Conference 77934 77956 LAW
Tuesday 77967 77974 DATE

 77975 77976 GPE
second 77983 77989 ORDINAL
0.5 seconds 78033 78044 TIME
3-point 78064 78071 CARDINAL
Stephen Curry 78085 78098 PERSON
third quarter 78128 78141 DATE
3-pointer 78170 78179 QUANTITY

 78244 78245 GPE
the last four seasons 78371 78392 DATE
12 78405 78407 CARDINAL
34-17 third quarter 78439 78458 DATE
Rockets 78481 78488 ORG

 78553 78554 GPE
Rockets 78628 78635 ORG
Mik

...

Kerrs 871908 871913 PERSON
fourth 871979 871985 ORDINAL
  872011 872012 NORP

 872036 872037 GPE
Casspi 872049 872055 LOC
10 or more minutes 872087 872105 TIME
only five 872106 872115 CARDINAL
January 872128 872135 DATE
3-pointers 872262 872272 CARDINAL
Golden State 872311 872323 ORG

 872403 872404 GPE
53 872456 872458 CARDINAL
Warrior 872470 872477 ORG
Casspi 872553 872559 PRODUCT
the Sacramento Kings 872578 872598 FAC
nine 3-pointers 872636 872651 CARDINAL
36 872664 872666 CARDINAL
Curry at Oracle Arena 872711 872732 ORG
Dec. 28, 2015 872736 872749 DATE

 872750 872751 GPE
last months 872766 872777 DATE
Kerr 872803 872807 PERSON
Golden State 872868 872880 GPE
11.9 873028 873032 CARDINAL
7.1 873044 873047 CARDINAL
seven 873073 873078 CARDINAL
nearly 59 percent 873101 873118 PERCENT

 873134 873135 GPE
Casspi 873193 873199 PERSON
12 minutes 873377 873387 TIME
12 873401 873403 CARDINAL
3s 873451 873453 CARDINAL

 873454 873455 GPE
Steve 873505 873510 PERSON

 873591 873592 GPE
Currys 8

112-99 894327 894333 DATE
Wednesday 894337 894346 DATE
night 894347 894352 TIME
15 894378 894380 CARDINAL
16 894388 894390 CARDINAL
the second half 894401 894416 DATE
11 894469 894471 CARDINAL
early in the fourth quarter 894488 894515 DATE
Magics 894534 894540 NORP

 894566 894567 GPE
Magic 894571 894576 EVENT
Payton 894636 894642 ORG
two 894652 894655 CARDINAL
Knicks 894680 894686 ORG
Porzingis 894704 894713 PERSON
three 894778 894783 CARDINAL

 894819 894820 GPE
Porzingiss 894823 894833 NORP
Tim Hardaway Jr. 894843 894859 PERSON
Knicks 894868 894874 ORG
26 894918 894920 CARDINAL
11 894932 894934 CARDINAL
38 minutes 894947 894957 TIME
Knicks 894963 894969 ORG
second 894970 894976 ORDINAL
Doug McDermott 894996 895010 PERSON
13 895020 895022 CARDINAL

 895044 895045 GPE
Knicks 895049 895055 ORG
six 895084 895087 CARDINAL
seven 895102 895107 CARDINAL
Porzingis 895163 895172 GPE
53 percent 895184 895194 PERCENT
23 895224 895226 CARDINAL

 895237 895238 GPE
Magic 895268 895273 ORG
54 perce

Porzingis 926412 926421 GPE
3-point 926436 926443 CARDINAL
fourth 926460 926466 ORDINAL

 926476 926477 GPE
2016 926480 926484 DATE
Bryan Colangelos 926486 926502 PERSON
first 926503 926508 ORDINAL
last year 926588 926597 DATE
Colangelo 926599 926608 PERSON
Sacramentos 926637 926648 ORG
2019 926658 926662 DATE
first 926663 926668 ORDINAL
Boston 926683 926689 GPE
two 926701 926704 CARDINAL
1 926718 926719 CARDINAL

 926832 926833 GPE
Philadelphia 926855 926867 GPE
Hinkie 926914 926920 PERSON
Noel 926933 926937 PERSON
Okafor   926942 926950 NORP

 927030 927031 GPE
76ers 927073 927078 DATE
Simmonss 927181 927189 NORP
last season 927226 927237 DATE
Fultz 927246 927251 PERSON

 927311 927312 GPE
Knicks 927337 927343 ORG
Porzingis 927521 927530 GPE
Knicks 927541 927547 ORG
Steve Mills 927608 927619 PERSON
Scott Perry 927624 927635 PERSON

 927636 927637 GPE
Porzingis 927672 927681 PERSON
  927726 927727 NORP
  927773 927774 NORP

 927856 927857 GPE
Porzingis 927885 927894 GPE
five-year 9278

Arseneault 953591 953601 ORG

 953643 953644 GPE
Arseneault 953644 953654 GPE
David Arseneault Sr 953680 953699 PERSON
the early 1990s 953710 953725 DATE
Grinnell 953747 953755 ORG
the last quarter-century 953931 953955 DATE

 953989 953990 GPE
one 953997 954000 CARDINAL
Grinnell 954020 954028 PRODUCT
David Arseneault Sr 954076 954095 PERSON
3-pointers 954151 954161 CARDINAL

 954271 954272 GPE
Rockets 954276 954283 ORG
3-pointers 954323 954333 CARDINAL
this season 954334 954345 DATE
3-point 954502 954509 CARDINAL
Nick Young 954558 954568 PERSON
Mondays 954630 954637 DATE
25-footer 954658 954667 CARDINAL

 954682 954683 GPE
three 954803 954808 CARDINAL
Arseneault 954816 954826 ORG

 954859 954860 GPE
Arseneault 954860 954870 GPE
Grinnell 954914 954922 ORG
Division III 954968 954980 ORG
one day 955051 955058 DATE
2014 955062 955066 DATE
Sacramento 955072 955082 GPE
David Arseneault Jr. 955128 955148 PERSON
the Reno Bighorns 955158 955175 ORG
the N.B.A. Development League 955196 955225 O

108 978885 978888 CARDINAL
Pacers 978910 978916 PERSON
Nov. 5 978920 978926 DATE
108-100 978930 978937 CARDINAL
Toronto 978952 978959 GPE
Nov. 22 978963 978970 DATE
28 978983 978985 CARDINAL
the third quarter 978995 979012 DATE
1991-92 979048 979055 DATE

 979056 979057 GPE
Knicks 979061 979067 ORG
Bankers Life Fieldhouse 979086 979109 ORG
8-27 979121 979125 DATE

 979126 979127 GPE
Pacers 979131 979137 PERSON
7-4 979147 979150 DATE
six 979181 979184 CARDINAL
Dec. 15 979232 979239 DATE
6 979250 979251 CARDINAL
120 979267 979270 CARDINAL
0-4 979282 979285 CARDINAL
less than 100 979299 979312 CARDINAL

 979313 979314 GPE
3 979476 979477 CARDINAL

 979530 979531 GPE
7 979574 979575 CARDINAL
Celtics 979614 979621 ORG
Rockets 979626 979633 ORG
3-point 979662 979669 CARDINAL

 979676 979677 GPE
Monday night 979680 979692 TIME
Rockets 979719 979726 NORP
7-for-44 979751 979759 CARDINAL
3 979765 979766 CARDINAL
16 percent 979771 979781 PERCENT
the entire season 979810 979827 DATE
26 percent 979

So this does a pretty good job but is also slow. Let's analysis how accurate the less precise textblob versions are:

In [27]:
team_names = pd.read_csv('../player_stats/team_names.csv')

Note that this data frame contains names of all teams along with number of wins this season and the round of playoffs.

In [28]:
team_names.head()

,Name,Wins,Playoffs,Conference,New York Team,City
0,Lakers,35,0,W,0,Los Angeles
1,Cavaliers,50,4,E,0,Cleveland
2,Warriors,58,5,W,0,Golden State
3,Celtics,55,3,E,0,Boston
4,Spurs,47,1,W,0,San Antonio


In [29]:
playoff_dict = {0:'DNQ',1:'First Round',2:'Second Round',3:'Conference Finals',4:'NBA Finals',5:'Champion'}

In [30]:
names = list(team_names['Name'])

Prediction with TextBlob tags:

In [31]:
for name in names:
    print('{} has {} occurrences in corpus'.format(name,c2[name.lower()]))

Lakers has 103 occurrences in corpus
Cavaliers has 370 occurrences in corpus
Warriors has 521 occurrences in corpus
Celtics has 302 occurrences in corpus
Spurs has 108 occurrences in corpus
76ers has 186 occurrences in corpus
Knicks has 590 occurrences in corpus
Bulls has 28 occurrences in corpus
Rockets has 296 occurrences in corpus
Raptors has 153 occurrences in corpus
Mavericks has 60 occurrences in corpus
Thunder has 67 occurrences in corpus
Suns has 41 occurrences in corpus
Heat has 75 occurrences in corpus
Hawks has 32 occurrences in corpus
Timberwolves has 46 occurrences in corpus
Hornets has 31 occurrences in corpus
Clippers has 99 occurrences in corpus
Pistons has 37 occurrences in corpus
Pacers has 91 occurrences in corpus
Blazers has 44 occurrences in corpus
Bucks has 62 occurrences in corpus
Wizards has 61 occurrences in corpus
Magic has 45 occurrences in corpus
Nets has 146 occurrences in corpus
Grizzlies has 29 occurrences in corpus
Kings has 52 occurrences in corpus
Nugg

In [32]:
team_names['noun_occurrences'] = [c2[name.lower()] for name in names]

Picking out these specific strings:

In [33]:
for name in names:
    print('{} has {} occurrences in corpus'.format(name,corpus.count(name)))

Lakers has 103 occurrences in corpus
Cavaliers has 369 occurrences in corpus
Warriors has 525 occurrences in corpus
Celtics has 297 occurrences in corpus
Spurs has 109 occurrences in corpus
76ers has 195 occurrences in corpus
Knicks has 591 occurrences in corpus
Bulls has 28 occurrences in corpus
Rockets has 296 occurrences in corpus
Raptors has 150 occurrences in corpus
Mavericks has 60 occurrences in corpus
Thunder has 77 occurrences in corpus
Suns has 42 occurrences in corpus
Heat has 71 occurrences in corpus
Hawks has 32 occurrences in corpus
Timberwolves has 52 occurrences in corpus
Hornets has 32 occurrences in corpus
Clippers has 100 occurrences in corpus
Pistons has 37 occurrences in corpus
Pacers has 90 occurrences in corpus
Blazers has 42 occurrences in corpus
Bucks has 58 occurrences in corpus
Wizards has 58 occurrences in corpus
Magic has 42 occurrences in corpus
Nets has 150 occurrences in corpus
Grizzlies has 29 occurrences in corpus
Kings has 52 occurrences in corpus
Nug

In [34]:
team_names['raw_occurrences'] = [corpus.count(name) for name in names]

In [35]:
team_names['% difference'] = ((team_names.raw_occurrences - team_names.noun_occurrences) / team_names.raw_occurrences)*100

In [36]:
team_names.sort_values(['noun_occurrences'],ascending=False)

,Name,Wins,Playoffs,Conference,New York Team,City,noun_occurrences,raw_occurrences,% difference
6,Knicks,29,0,E,1,New York,590,591,0.169205
2,Warriors,58,5,W,0,Golden State,521,525,0.761905
1,Cavaliers,50,4,E,0,Cleveland,370,369,-0.271003
3,Celtics,55,3,E,0,Boston,302,297,-1.683502
8,Rockets,65,3,W,0,Houston,296,296,0.000000
5,76ers,52,2,E,0,Philadelphia,186,195,4.615385
9,Raptors,59,2,E,0,Toronto,153,150,-2.000000
24,Nets,28,0,E,1,Brooklyn,146,150,2.666667
4,Spurs,47,1,W,0,San Antonio,108,109,0.917431
29,Pelicans,48,2,W,0,New Orleans,106,104,-1.923077


So from this analysis we find that there is little discrepancy between the number of raw strings matching the team names in the text and the number found by restricting ourselves to TextBlob nouns. This seems like a sensible way to proceed with our analysis.

In [38]:
team_names.to_csv('NER_vs_raw_strings.csv',index=False)

# 4. Comparing Sentiment Classifiers:

Read in files:

In [39]:
articles_list = pd.read_csv('../nyt_scrape/articles_list_w_date.csv',parse_dates=['date'])

In [40]:
file_title = articles_list.article_urls.apply(lambda x : x.replace("/","").replace(".",""))
article_date = articles_list.date

title_date_dict = dict(zip(file_title,article_date))

In [42]:
title_text_dict = dict()

for article in file_title:
    with open(f'../nyt_scrape/articles/{article}.txt') as f:
        title_text_dict[article] = f.read().replace('’','').replace('”','').replace('“','').replace('—','').split('\n')

In [43]:
para = TextBlob(title_text_dict[file_title[0]][0])

In [44]:
print(para, para.sentiment)

The Golden State Warriors have ruined basketball. Sentiment(polarity=0.3, subjectivity=0.5)


At first glance this sentiment analysis looks to be quite poor - we would hope this paragraph would register negative sentiment.

**Vader Sentiment:**

In [46]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [47]:
vader = SentimentIntensityAnalyzer()

In [48]:
vader.polarity_scores(para)

{'neg': 0.341, 'neu': 0.659, 'pos': 0.0, 'compound': -0.4767}

In [49]:
for para in title_text_dict[file_title[25]][:5]:
    print(f'{para}\nTextBlob Sentiment Scores:{TextBlob(para).sentiment}\nVader Sentiment Scores:{vader.polarity_scores(para)}\n\n')

BOSTON  With a little help from his friends, though not that much, LeBron James dazzled once again in a playoff elimination game, sending the Cavaliers back to the N.B.A. finals on Sunday night and keeping himself in a Cleveland uniform for at least another two weeks.
TextBlob Sentiment Scores:Sentiment(polarity=-0.1375, subjectivity=0.3)
Vader Sentiment Scores:{'neg': 0.0, 'neu': 0.882, 'pos': 0.118, 'compound': 0.6712}


In a game that looked bleak at the outset, with the Cavaliers playing without the injured Kevin Love, James powered an offense that started the game atrociously and then did just enough to break the upstart hearts of the young Boston Celtics, 87-79, in Game 7 of the Eastern Conference finals.
TextBlob Sentiment Scores:Sentiment(polarity=-0.2875, subjectivity=0.5875)
Vader Sentiment Scores:{'neg': 0.036, 'neu': 0.814, 'pos': 0.149, 'compound': 0.7398}


James played all 48 minutes and finished with 35 points, 15 rebounds and 9 assists, missing a triple-double only bec

On balance, Vader Sentiment appears to be more accurately capturing the sentiment of the article.